In [8]:
%matplotlib inline
import os
import sys
import itertools
import re
import pprint
import imp

import xray
import numpy as np
import pandas as pd

sys.path.append('/nuwa_cluster/home/jackyu/climatools/')
import climatools.misc as climamisc
imp.reload(climamisc)

from IPython.display import HTML

# Subroutine relations

In [27]:
def fortran_files():
    dir_cam = '/nuwa_cluster/home/jackyu/climate_models/cesm1_2_2/models/atm/cam/src/physics/cam/'
    cam_files = ['aer_rad_props.F90', 
                 'modal_aer_opt.F90',
                 'rad_constituents.F90', 
                 'phys_prop.F90', 
                 'physpkg.F90']
    fpaths = [os.path.join(dir_cam, file) for file in cam_files]
    return fpaths

In [3]:
#pprint.pprint(climamisc.Fortran_subroutine_relations_from_files(paths_fortran = fortran_files()))

# Aerosol data files

Data for certain aerosol optical properties are read in from the following files during initialisation (so this is done only once at the start of each model run), by the subroutine physprop_init().  Shown here are the name of the file, the optics method and the file index.  

In [1]:
def aer_dir():
    return '/nuwa_data/data/cesm1/inputdata/atm/cam/physprops/'

def nc_files_read_in_physprop_init():
    print( '(file name, optics method, fileindex)' )
    filelist = [('mam3_mode1_rrtmg_c110318.nc', 'modal', 1),
                ('mam3_mode2_rrtmg_c110318.nc', 'modal', 2),
                ('mam3_mode3_rrtmg_c110318.nc', 'modal', 3),
                ('sulfate_rrtmg_c080918.nc', 'hygroscopic', 4),
                ('ocpho_rrtmg_c101112.nc', 'insoluble', 5),
                ('ocphi_rrtmg_c100508.nc', 'hygroscopic', 6),
                ('bcpho_rrtmg_c100508.nc', 'insoluble', 7),
                ('dust4_rrtmg_c090521.nc', 'insoluble', 8),
                ('ssam_rrtmg_c100508.nc', 'hygroscopic', 9)]
    return filelist

In [6]:
# mam3_mode1
def print_mam3mode1():
    print('mam3 mode1')

    print('''
    Note that because Fortran fills its arrays' elements with a different order, the order of the dimensions
    displayed here by xray are opposite of those in the Fortran code --- transpose.
    ''')

    with xray.open_dataset(
        os.path.join(aer_dir(), 'mam3_mode1_rrtmg_c110318.nc')) as ds:
        #print(ds)
        print()
        print('extpsw')
        print(ds['extpsw'].attrs)
        print()
        print('abspsw')
        print(ds['abspsw'].attrs)
        print()
        print('asmpsw')
        print(ds['asmpsw'].attrs)
        print()
        print('sigma_logr_aer')
        print(ds['sigmag'].attrs)
        print()
        print('refrtabsw')
        print(ds['refindex_real_sw'].attrs)
        print()
        print('refitabsw')
        print(ds['refindex_im_sw'].attrs)
        
        
def print_bcpho():
    print('bcpho_rrtmg_c100508.nc')

    with xray.open_dataset(os.path.join(aer_dir(), 'bcpho_rrtmg_c100508.nc')) as ds:
        print(ds)
#        print()
#        print('density_aer')
#        print(ds['density'].attrs)
#        print()
#        print('hygro_aer')
#       print(ds['hygroscopicity'].attrs)
#       print()
        

In [2]:
nc_files_read_in_physprop_init()

(file name, optics method, fileindex)


[('mam3_mode1_rrtmg_c110318.nc', 'modal', 1),
 ('mam3_mode2_rrtmg_c110318.nc', 'modal', 2),
 ('mam3_mode3_rrtmg_c110318.nc', 'modal', 3),
 ('sulfate_rrtmg_c080918.nc', 'hygroscopic', 4),
 ('ocpho_rrtmg_c101112.nc', 'insoluble', 5),
 ('ocphi_rrtmg_c100508.nc', 'hygroscopic', 6),
 ('bcpho_rrtmg_c100508.nc', 'insoluble', 7),
 ('dust4_rrtmg_c090521.nc', 'insoluble', 8),
 ('ssam_rrtmg_c100508.nc', 'hygroscopic', 9)]

Optics method determines what optical properties are imported.  The file index is used to access the optical properties during model run and is in fact just the order in which the files are read during initialisation.  For example, mam3_mode3_rrtmg_c110318.nc is the third file to be opened and read, so it has a file index of 3.  

The data imported from these files are all saved in the *physprop* object created during initialisation. So, physprop(1) points to data imported from the file mam3_mode1_rrtmg_c110318.nc, while physprop(7) points to data imported from file bcpho_rrtmg_c100508.nc.

Data stored in physprop are needed when computing aerosol optical properties that are to be used in radiation calculations.  They are constant in time and so do not change with model's nstep.  
*Which ones* and *How* they are used (for example, how they are combined with other dynamic factors at each nstep) depends on the aerosol model chosen. 

As an example, the content of bcpho_rrtmg_c100508.nc is:

In [9]:
print_bcpho()

bcpho_rrtmg_c100508.nc
<xray.Dataset>
Dimensions:                      (lw_band: 16, near-ir_band: 44, rh_idx: 1, sw_band: 14)
Coordinates:
  * lw_band                      (lw_band) int64 0 1 2 3 4 5 6 7 8 9 10 11 ...
  * near-ir_band                 (near-ir_band) int64 0 1 2 3 4 5 6 7 8 9 10 ...
  * rh_idx                       (rh_idx) int64 0
  * sw_band                      (sw_band) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13
Data variables:
    refindex_real_aer_sw         (sw_band) >f8 1.95 1.95 1.95 1.95 1.95 1.95 ...
    refindex_im_aer_sw           (sw_band) >f8 -0.79 -0.79 -0.79 -0.79 -0.79 ...
    hygroscopicity               >f8 1e-10
    asm_sw                       (sw_band, rh_idx) >f8 0.0465 0.06675 ...
    dryrad                       >f8 1.18e-08
    refindex_real_aer_lw         (lw_band) >f8 1.95 1.95 1.95 1.95 1.95 1.95 ...
    ext_sw                       (sw_band, rh_idx) >f8 792.7 999.8 1.222e+03 ...
    opticstype                   |S12 b'insoluble   '
    num_to_m

Under Data Variables, the value for opticsmethod is 'insoluble', so the data from this file is imported by the insoluble_optics_init() subroutine.  Most data are imported as is, but some might be transformed before it is saved in physprop.  Some of the variables might be given a different name in physprop, even though the content, or values, are exactly the same.

# Modal aerosol model 3 (MAM3)
By default in CESM 1.2.2, modal aerosol model 3 (MAM3) is used in preparing aerosol optical properties for radiation calculations.  Another option is the *bulk* model.

The constituents and structure of this model can be viewed in this [tree diagram][mam3_tree].

The model consists of three modes.  Each mode can be divided in to a number-mixing part and a mass-mixing part.  Each mass-mixing part can consist of several aerosol species, such as *sulfate*, *so4*, etc.  Source of aerosols, whether number-mixing or mass-mixing, and of whatever species, can be either *interstitial* or *cloud-bourne*; the current choice is interstitial.


[mam3_tree]: http://bl.ocks.org/qAp/raw/1ebccc6fdf2d0313482d/

# The *modes* object

*modes* is a data structure that holds information parsed in from the &rad_cnst_nl namelist. In this case, the model described by &rad_cnst_nl namelist is MAM3.  Part of the *modes* object for this looks like [this][modes_MAM3]. 

The following glossary/comment from subroutine parse_mode_defs() explains what each term means.


   ! 'mode_name:mode_type:=',                                                                                                
   !  'source_num_a:camname_num_a:source_num_c:camname_num_c:num_mr:+',                                                      
   !  'source_mmr_a:camname_mmr_a:source_mmr_c:camname_mmr_c:spec_type:prop_file[:+]'[,]                                     
   !  ['source_mmr_a:camname_mmr_a:source_mmr_c:camname_mmr_c:spec_type:prop_file][:+][']                                    
   !
   ! where the ':' separated fields are:                                                                                     
   ! mode_name -- name of the mode.                                                                                          
   ! mode_type -- type of mode.  Valid values are from the MAM code.                                                         
   ! =         -- this line terminator identifies the initial string in a                                                    
   !              mode definition                                                                                            
   ! +         -- this line terminator indicates that the mode definition is                                                 
   !              continued in the next string                                                                               
   ! source_num_a  -- Source of interstitial number mixing ratio,  'A', 'N', or 'Z'                                          
   ! camname_num_a -- the name of the interstitial number component.  This name must be                                      
   !                  registered in the constituent arrays when source=A or in the                                           
   !                  physics buffer when source=N                                                                           
   ! source_num_c  -- Source of cloud borne number mixing ratio,  'A', 'N', or 'Z'                                           
   ! camname_num_c -- the name of the cloud borne number component.  This name must be                                       
   !                  registered in the constituent arrays when source=A or in the                                           
   !                  physics buffer when source=N                                                                           
   ! source_mmr_a  -- Source of interstitial specie mass mixing ratio,  'A', 'N' or 'Z'                                      
   ! camname_mmr_a -- the name of the interstitial specie.  This name must be                                                
   !                  registered in the constituent arrays when source=A or in the                                           
   !                  physics buffer when source=N                                                                           
   ! source_mmr_c  -- Source of cloud borne specie mass mixing ratio,  'A', 'N' or 'Z'                                       
   ! camname_mmr_c -- the name of the cloud borne specie.  This name must be                                                 
   !                  registered in the constituent arrays when source=A or in the                                           
   !                  physics buffer when source=N                                                                           
   ! spec_type -- species type.  Valid values far from the MAM code, except that                                             
   !              the value 'num_mr' designates a number mixing ratio and has no                                             
   !              associated field for the prop_file.  There can only be one entry                                           
   !              with the num_mr type in a mode definition.                                                                 
   ! prop_file -- For aerosol species this is a filename, which is                                                           
   !              identified by a ".nc" suffix.  The file contains optical and                                               
   !              other physical properties of the aerosol. 
   !
   ! A mode definition must contain only 1 string for the number mixing ratio components                                     
   ! and at least 1 string for the species. 
   
   
   *modes* does not hold actual data on aerosol properties, rather it holds information about where to look for those data, and these data can be static data, like those stored in the *physprop* object, or dynamic ones such as aerosol mass-mixing ratio.
   
   The source_ variables in *modes* indicates where to look for the dynamic variables.  'A' means to look in the physical buffer; 'Z' means to look in model state. 
   
   [modes_MAM3]: http://bl.ocks.org/qAp/raw/e45416c5a722ddb8042e/

# Subroutine: modal_aero_sw()

subroutine modal_aero_sw() in modal_aer_opt.F90

In this subroutine aerosol properties that change with time and those that do not change with time are combined to form aerosol optical properties that can be used by RRTMG directly, namely: optical depth, single scattering albedo, asymmetry factor and forward scattering factor.  In the pseudo-code below, the dimensions of each variable indicated, and wherever possible, the units as well. 


Dimensions:

+ pcols --- number of columns/soundings
+ pver --- number of model/atm layers
+ nswbands --- number of shortwave spectral bands (= 14)
+ ncoef --- number of coeffiecients (= 5)
+ prefr --- number of real refractive indices (= 7)
+ prefi --- number of imaginary refractive indices (= 10)

Notes:

+ Static variables are coloured green.
+ Dimension indicators are read as such.  For example, *specmmr(pcols, pver: m, l)*. *specmmr* is an array of dimensions *pcols* x *pver* and it corresponds to aerosol mode *m* and species *l*.

---------------------------------------------------------------------
Pseudo code

+ Get layer mass, *mass(pcols, pver)*, from model state.
+ Get air density, *air_density(pcols, pver)*, from model state.
+ Get number of modes, *nmodes*, for *list_idx*.
+ Get number mode wet diametre for all modes, *dgnumwet_m(pcols, pver, nmodes)* and aerosol water for all modes, *qaerwat_m(pcols, pver, nmodes)[g/g]*, from physical buffer. 



+ **Start loop over modes. do m = 1, *nmodes***
+ Get the following properties from initialised table, *physprop*:

   * geometric standard deviation of number distribution, <span style="color: green">*sigma_logr_aer(: m)*</span>
   * table of real refractive indices for aerosols, <span style="color: green">*refrtabsw(prefr, nswbands: m)*</span>
   * table of imag refractive indices for aerosols, <span style="color: green">*refitabsw(prefi, nswbands: m)*</span>
   * specific extinction, <span style="color: green">*extpsw(ncoef, prefr, prefi, nswbands: m)[m2 kg-1]*</span>
   * specific absorption, <span style="color: green">*abspsw(ncoef, prefr, prefi, nswbands: m)[m2 kg-1]*</span>
   * asymmetry factor, <span style="color: green">*asmpsw(ncoef, prefr, prefi, nswbands: m)[m2 kg-1]*</span>
   
+ Get the number species, *nspec(: m)*, from initialised table, *modes*.
+ Compute the following size parametres using *pcols*, *sigma_logr_aer(: m)* and *dgnumwet(pcols, pver: m)*

   * aerosol surface mode radius, *radsurf(pcols, pver: m)*
   * log aerosol surface mode radius, *logradsurf(pcols, pver: m)*
   * *cheb(ncoef, pcols, pver: m)*
   


+ **Start loop over spectral bands. do isw = 1, nswbands**
+ **Start loop over layers. do k = top_lev, pver**
+ **Start loop over aerosol species of this mode. do l = 1, nspec**


+ Get species mass mixing ratio, *specmmr(pcols, pver: m, l)*, from either *state* or *pbuf*.
+ Get the following from initialised table *physprop*:

   * species density, <span style="color: green">*specdens(: m, l)[kg/m3]*</span>
   * species refractive index, <span style="color: green">*specrefindex(nswbands: m, l)*</span>
   * species type, <span style="color: green">*spectype(: m, l)*</span>
   * <span style="color: green">*hygro_aer(: m, l)*</span>
   
+ Compute volume concentration of aerosol, *vol(pcols: m, l, k)* from *specmmr(pcols, pver: m, l)* and *specdens(: m, l)[kg/m3]*.
+ Add above to volume concentration of aerosol for all species, *dryvol(pcols: m, k)*.
+ Compute complex refractive index for this species from *vol(pcols: m, l, k)* and *specrefindex(nswbands: m, l)* and add to complex refractive index for all species, *crefin(pcols: m, k, iw)*.
+ **End loop over aerosol species of this mode.**


+ Compute volume concentration of water, *watervol(pcols: k, m)*, from *qaerwat(pcols, pver: m)* and *rhoh20*.
+ Compute volume concentration of wet, *wetvol(pcols: k, m)[m3/kg]*.
+ Use *wetvol(pcols: k, m)[m3/kg]* and complex refractive index for water visible,  *crefwsw(nswbands)*, to change *crefin(pcols: m, k, isw)*.
+ Split *crefin(pcols: m, k, isw)* into real and imaginary parts, *refr(pcols: m, k, isw)* and *refi(pcols: m, k, isw)*.


+ Compute, using bilinear interpolation, *cext(pcols, ncoef: m, k, isw)*, from *extpsw(ncoef, prefr, prefi, nswbands)*, *refr(pcols: m, k, isw)*, *refi(pcols: m, k, isw)*, *refrtabsw(prefr, nswbands: m)* and *refitabsw(prefi, nswbands: m)*.
+ Compute, using bilinear interpolation, *cabs(pcols, ncoef: m, k, isw)*, from *abspsw(ncoef, prefr, prefi, nswbands)*, *refr(pcols: m, k, isw)*, *refi(pcols: m, k, isw)*, *refrtabsw(prefr, nswbands: m)* and *refitabsw(prefi, nswbands: m)*.
+ Compute, using bilinear interpolation, *casm(pcols, ncoef: m, k, isw)*, from *asmpsw(ncoef, prefr, prefi, nswbands)*, *refr(pcols: m, k, isw)*, *refi(pcols: m, k, isw)*, *refrtabsw(prefr, nswbands: m)* and *refitabsw(prefi, nswbands: m)*.


+ Compute parameterised specific extinction, *pext(pcols: m, k, isw)* from *cext(pcols, ncoef: m, k, isw)* and *cheb(ncoef, pcols, pver: m)*
+ Compute parameterised specific extinction, *pext(pcols: m, k, isw)[m2/kg aerosol]*, from *pext(pcols: m, k, isw)[m2/kg water]*, *wetvol(pcols: k, m)[m3/kg]* and *rhoh20*.
+ Compute parameterised specific absorption, *pabs(pcols: m, k, isw)[m2/kg]*, from *cabs(pcols, ncoef: m, k, isw)* and *cheb(ncoef, pcols, pver: m)*.
+ Compute parameterised asymmetry factor, *pasm(pcols: m, k, isw)*, from *casm(pcols, ncoef: m, k, isw)* and *cheb(ncoef, pcols, pver: m)*.
+ Adjust *pabs(pcols: m, k, isw)* using *wetvol(pcols: k, m)[m3/kg]* and *rhoh20*.



+ Compute parameterised single scattering albedo, *palb(pcols: m, k, isw)*, from *pabs(pcols: m, k, isw)* and *pext(pcols: m, k, isw)*. <span style="color: red">repeated in the code</span>
+ Compute aerosol optical depth, *dopaer(pcols: m, k, isw)*, from *pext(pcols: m, k, isw)[m2/kg aerosol]* and *mass(pcols, pver)*.



+ Add to aerosol optical depth, *tauxar(pcols, k, isw)*, *dopaer(pcols: m, k, isw)*.
+ Add to aerosol single scattering albedo, *wa(pcols, k, isw)*,  *dopaer(pcols: m, k, isw) x palb(pcols: m, k, isw)*.
+ Add to aerosol asymmetry factor, *ga(ncol, k, isw)*, *dopaer(pcols: m, k, isw) x palb(pcols: m, k, isw) x pasm(pcols: m, k, isw)*.
+ Add to forward scattering factor, *fa(ncol, k , isw)*, *dopaer(pcols: m, k, isw) x palb(pcols: m, k, isw) x pasm(pcols: m, k, isw) x pasm(pcols: m, k, isw)*.


+ **end loop over layers**
+ **end loop over spectral bands**
+ **end loop over nmodes**

-----------------------------------------------------------------------------------------------------------------

More verbosely...

The aim is to get the total optical property over all modes.

Each mode has a *wet diametre* and a *geometric standard deviation of number distribution*.  These are used to compute size parametres for the mode, including one named *cheb*.

Each mode also contains several aerosol species.  For each species, the volume concentration is computed from the species' mass-mixing ratio and density.  This is then combined with the species' refractive index to form the *complex refractive index* for the species.  The complex refractive index from all species are added together to give a total for the mode. 

Next, and in a similar fashion, the contribution from water and wet's refractive index is added to this complex refractive index.

The complex refractive index is split into real and imaginary parts. 

Now, separately, for each mode as a whole, there is a corresponding refractive index (*mode refractive index*),  as well as optical properties: specific extinction, specific absorption and asymmetry factor, which are functions of the refractive index.  Using bilinear interpolation, in conjunction with the real and imaginary parts of both the complex refractive index and the mode refractive index, each of these optical properties are turned into counterparts that are no longer functions of the refractive index.

Next, these optical properties are combined with the mode's size parametre *cheb* to form *parameterised* versions: parameterised specific extinction, parameterised specific absoprtion and parameterised asymmetry factor.

The parameterised specific extinction and absorption are adjusted to take into account contribution from water.

Next, parameterised extinction and absoprtion combine to give parameterised single scattering albedo, and parameterised extinction combine with the layer mass to give parameterised optical depth.  

The parameterised optical depth is identified as the aerosol optical depth of the mode.
The product of the parameterised optical depth and the parameterised single scattering albedo is identified as the aerosol single scattering albedo of the mode.
The product of the parameterised optical depth, the parameterised single scattering albedo and the parameterised asymmetry factor is identified as the aerosol asymmetry factor of the mode.
The product of the aerosol asymmetry factor of the mode and the parameterised asymmetry factor is identified as the aerosol forward scattering factor of the mode.


The total aerosol optical depth, single scattering albedo, asymmetry factor and forward scattering factor are obtained by summing over all modes.  These are input to RRTMG.







In [43]:
HTML('''
<script>
show_code = true;
function code_toggle() {
if (show_code) {$('div.input').hide();} else {$('div.input').show();}
show_code = !show_code
}
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code"></form>
''')